In [ ]:
import pandas as pd

In [ ]:
targets = {'Ewing sarcoma': '../ewing/data/TH-Ewing-Pathways-Enrichment.tsv',
            'MYCN-NA Neuroblastoma': '../nbl/mycn-na/data/TARGET-MYCN-NA-Pathways-Enrichment.tsv',
            'Osteosarcoma': '../osteo/treehouse/data/TH-Osteo-Pathways-Enrichment.tsv',
            'eRMS': '../rhabdo/data/TH-eRMS-Pathways-Enrichment.tsv',
            'aRMS': '../rhabdo/data/TH-aRMS-Pathways-Enrichment.tsv',
            'Synovial sarcoma': '../synovial/data/micro-58-synovial-Pathways-Enrichment.tsv'}

In [ ]:
theme = {'Osteosarcoma (cluster 0)': 'Osteosarcoma (Regulation of Translation)',
         'Osteosarcoma (cluster 1)': 'Osteosarcoma (Wound Healing)',
         'Osteosarcoma (cluster 2)': 'Osteosarcoma (Innate Immune)',
         'Osteosarcoma (cluster 3)': 'Osteosarcoma (Complement/Humoral Immune)',
         'Ewing sarcoma (cluster 0)': 'Ewing sarcoma (Cell Cycle)',
         'Ewing sarcoma (cluster 1)': 'Ewing sarcoma (rRNA Processing)',
         'Ewing sarcoma (cluster 2)': 'Ewing sarcoma (Wound Healing)',
         'Ewing sarcoma (cluster 3)': 'Ewing sarcoma (Innate/Adaptive Immune)',
         'Ewing sarcoma (cluster 4)': 'Ewing sarcoma (Myogenesis)',
         'MYCN-NA Neuroblastoma (cluster 0)': 'MYCN-NA Neuroblastoma (Adaptive Immune)',
         'MYCN-NA Neuroblastoma (cluster 1)': 'MYCN-NA Neuroblastoma (Regulation of Translation)',
         'MYCN-NA Neuroblastoma (cluster 2)': 'MYCN-NA Neuroblastoma (Wound Healing)',
         'eRMS (cluster 0)': 'Embryonal Rhabdomyosarcoma (Regulation of Translation)',
         'eRMS (cluster 1)': 'Embryonal Rhabdomyosarcoma (Adaptive Immune)',
         'aRMS (cluster 0)': 'Alveolar Rhabdomyosarcoma (Epithelial)',
         'aRMS (cluster 1)': 'Alveolar Rhabdomyosarcoma (Adaptive Immune)',
         'aRMS (cluster 2)': 'Alveolar Rhabdomyosarcoma (mRNA Processing)',
         'Synovial sarcoma (cluster 0)': 'Synovial sarcoma (Regulation of Translation)',
         'Synovial sarcoma (cluster 1)': 'Synovial sarcoma (Innate Immune)',
         'Synovial sarcoma (cluster 2)': 'Synovial sarcoma (Neuronal)', 
         'Synovial sarcoma (cluster 3)': 'Synovial sarcoma (Cell Cycle)'}

In [ ]:
pd.read_csv('../nbl/mycn-na/data/TARGET-MYCN-NA-Pathways-Enrichment.tsv', sep='\t')

In [ ]:
sigs = set()

for disease, target in targets.items():
    print(disease)
    df = pd.read_csv(target, sep='\t', index_col=[0], header=[0, 1])
    for cluster, t in df.groupby(level=0, axis=1):
        t = t[cluster]
        hits = t[(t['NES'] > 0) & (t['padj'] < 0.05)].sort_values('NES', ascending=False).head(10).index.values
        sigs.update(hits)

In [ ]:
plot_df = pd.DataFrame(index=sigs)

for disease, target in targets.items():
    df = pd.read_csv(target, sep='\t', index_col=[0], header=[0, 1])
    for cluster, t in df.groupby(level=0, axis=1):
        tag = '%s (cluster %s)' % (disease, cluster)
        tag = theme[tag]
        for p in plot_df.index.values:
            try:
                padj = t[cluster].loc[p, 'padj']
                if padj < 0.05:
                    NES = t[cluster].loc[p, 'NES']
                
                else:
                    NES = 0.0
            
            except KeyError:
                NES = 0.0
                
            plot_df.loc[p, tag] = NES

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.clustermap(plot_df,
               cmap=sns.diverging_palette(240, 10, n=7))

pth = 'img/pan-cancer-NES-heatmap.svg'
plt.savefig(pth, format='svg', bbox_inches='tight')

In [ ]:
sns.clustermap(plot_df,
               cmap=sns.diverging_palette(240, 10, n=7),
               figsize=(10, 40))

pth = 'img/pan-cancer-NES-heatmap-long.svg'
plt.savefig(pth, format='svg', bbox_inches='tight')